In [1]:
# add modules folder to Python's search path
import os, sys
from pathlib import Path

script_dir = Path(os.path.abspath(''))
module_dir = str(script_dir.parent)
print(script_dir)
print(module_dir)
sys.path.insert(0, module_dir + '/modules')
# import necessary modules
import wasserstein as ws
import tensorflow_probability as tfp
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\pinak\Documents\GitHub\sinkhorn-descent\notebooks
C:\Users\pinak\Documents\GitHub\sinkhorn-descent


**Test Sinkhorn algorithm with near-zero weights**

In [21]:
dimension, n_particles, low_weight = 10, 50, 1e-500
half = int(n_particles/2)
particles_1 = tf.random.normal(shape=(n_particles, dimension)) + tf.ones((n_particles, dimension))
weights_1 = [low_weight for _ in range(int(n_particles/2))] + [(1- 0.5 * n_particles * low_weight)/(0.5 * n_particles) for _ in range(int(n_particles/2))]
particles_2 = tf.random.normal(shape=(n_particles, dimension))
weights_2 = [1.0/n_particles for _ in range(n_particles)]

# Sinkhorn with small weights
print(ws.sinkhorn_div_tf(particles_1, particles_2, weights_1, weights_2)[0]);
# Sinkhorn without small weights
print(ws.sinkhorn_div_tf(particles_1[half:], particles_2[half:], weights_1[half:], weights_2[half:])[0]);

tf.Tensor(17.59238, shape=(), dtype=float32)
tf.Tensor(18.32245, shape=(), dtype=float32)


**Check if Sinkhorn Gradent descent is working properly**

In [13]:
_, grads = ws.sink_grad_position(particles_1[half:], particles_2[half:], weights_1[half:], weights_2[half:]);
positions = [tf.Variable(z) for z in particles_1[half:]]
optimizer = tf.keras.optimizers.Adam(1e-1)
optimizer.apply_gradients(zip(grads, positions),  experimental_aggregate_gradients=False)
# output should be a positive number
ws.sinkhorn_div_tf(particles_1[half:], particles_2[half:], weights_1[half:], weights_2[half:])[0]-ws.sinkhorn_div_tf(positions, particles_2[half:], weights_1[half:], weights_2[half:])[0]

<tf.Tensor: shape=(), dtype=float32, numpy=2.1528301>

**Define weight function, pdf and sampler**

In [6]:
def w(x, y):
  return tf.math.exp(-tf.reduce_sum((x-y)**2, axis=1))
y = tf.ones((n_particles, dimension))

X = tfp.distributions.MultivariateNormalDiag(scale_diag=tf.ones(dimension))
def p(x):
  return X.prob(x)
def sampler(shape):
  return X.sample(shape)

**Use Sinkhorn Gradient descent to find a uniform sample**

In [7]:
sample = sampler(n_particles)
weights = w(sample, y)
usf = ws.UniformSampleFinder(sample, weights, cost_p=1)
usf.find();

Step = 1, Sinkhorn divergence = 6.46921968460083
Step = 2, Sinkhorn divergence = 5.7132697105407715
Step = 3, Sinkhorn divergence = 5.328889846801758
Step = 4, Sinkhorn divergence = 4.822479724884033
Step = 5, Sinkhorn divergence = 4.345849990844727
Step = 6, Sinkhorn divergence = 3.928879976272583
Step = 7, Sinkhorn divergence = 3.580709934234619
Step = 8, Sinkhorn divergence = 3.2697598934173584
Step = 9, Sinkhorn divergence = 2.9816298484802246
Step = 10, Sinkhorn divergence = 2.7659800052642822
Step = 11, Sinkhorn divergence = 2.582059860229492
Step = 12, Sinkhorn divergence = 2.4069299697875977
Step = 13, Sinkhorn divergence = 2.2512099742889404
Step = 14, Sinkhorn divergence = 2.1159698963165283
Step = 15, Sinkhorn divergence = 1.993209958076477
Step = 16, Sinkhorn divergence = 1.8995800018310547
Step = 17, Sinkhorn divergence = 1.8292399644851685
Step = 18, Sinkhorn divergence = 1.75232994556427
Step = 19, Sinkhorn divergence = 1.6701699495315552
Step = 20, Sinkhorn divergence =

In [8]:
def test(z):
  return tf.reduce_sum(z**2)

z = tf.convert_to_tensor([1, 2, 0., 6.])
with tf.GradientTape() as tape:
  tape.watch(z)
  f = test(z)
tape.gradient(f, z)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 2.,  4.,  0., 12.], dtype=float32)>

In [26]:
import numpy as np
u_sample = np.array([t.numpy() for t in usf.u_sample])
u_sample.shape

(50, 10)

In [27]:
np.exp(-100)

3.720075976020836e-44